In [1]:
import numpy as np

In [2]:
# initialize threshold/capacity c

c = 4 # in all queues, we make sure that there are no more than c customers at the end of each time interval [t - 1, t]

def elementWiseMin(a1, a2):
    output = np.array([0, 0, 0])
    for i in range(3):
        output[i] = min(a1[i], a2[i])
    return output

# initialize average arrival rates

a1_avg = 0.3 # average arrival rate for queue 1 (input 1 to output 2)
a2_avg = 0.3 # average arrival rate for queue 2 (input 2 to output 1)
a3_avg = 0.69 # average arrival rate for queue 3 (input 2 to output 2)

def anArrivalVector(a1_avg, a2_avg, a3_avg):
    return np.array([np.random.poisson(a1_avg), np.random.poisson(a2_avg), np.random.poisson(a3_avg)])

def elementWiseMax(a1, a2):
    output = np.array([0, 0, 0])
    for i in range(3):
        output[i] = max(a1[i], a2[i])
    return output

# Estimate the values $v$ of the states $\vec{q}$ under policy 1 (maxWeight)

In [3]:
# estimate the values v of the states q under policy 1 (maxWeight)

def chooseMaxWeightServiceConfiguration(s, q):
    if q[0] + q[1] >= q[2]:
        s[0] = s[1] = 1
        return
    s[2] = 1

# initialize current states q, past average reward m, and value estimates v at time t = 0

q = np.array([0, 0, 0])
m = 0
v = np.zeros((c + 1, c + 1, c + 1))

# update v 10,000,000 times (with the first one being trivial)

for t in range(1, 10000001):
    # right now we've got q(t - 1)
    s = np.array([0, 0, 0])
    chooseMaxWeightServiceConfiguration(s, q) # compute service configuration s(t - 1) for the interval [t - 1, t]
    a = anArrivalVector(a1_avg, a2_avg, a3_avg) # generate arrival configuration a(t - 1) for the interval [t - 1, t]
    r = sum(q) # compute current reward r(t - 1)
    m_t_minus_1 = m # store m(t - 1)
    m = (1 - 1 / t) * m + (1 / t) * r # compute past average reward m(t) based on t, m(t - 1), and current reward r(t - 1)
    q_t_minus_1 = q # store q(t - 1)    
    q = elementWiseMin(elementWiseMax(q + a - s, np.array([0, 0, 0])), np.array([c, c, c])) # compute current state q(t)
    
    # update value estimate v(q(t - 1)) based on v(q(t - 1)), t, r(t - 1), m(t - 1), and v(q(t))
    v[q_t_minus_1[0], q_t_minus_1[1], q_t_minus_1[2]] += (1 / t) * (r - m_t_minus_1 + v[q[0], q[1], q[2]] - v[q_t_minus_1[0], q_t_minus_1[1], q_t_minus_1[2]])

In [4]:
print('For maxWeight, the state-value estimates are summarized in')
print('v =\n', v, '\n')

For maxWeight, the state-value estimates are summarized in
v =
 [[[-2.47056969e+00 -2.21383351e+00 -1.30474699e+00 -3.00307022e-01
    1.32831055e-01]
  [-2.74487439e-01 -7.62950897e-01 -3.69093025e-01  2.24174280e-01
    3.58285945e-01]
  [-3.38080518e-02 -8.22869090e-02  1.08597718e-01  3.71459687e-01
    7.30122235e-01]
  [-1.63329156e-03 -6.80675407e-04  7.22980856e-02  3.76243346e-01
    8.90208689e-01]
  [-1.05746876e-05  1.32745190e-03  2.37450251e-02  1.36673948e-01
    2.62177520e-01]]

 [[-3.18756939e-01 -8.34359154e-01 -4.44401582e-01  1.78536030e-01
    3.20353824e-01]
  [-6.18776476e-02 -1.12418528e-01 -4.44593563e-02  3.72613112e-01
    7.16645362e-01]
  [ 3.52925013e-01 -5.46390022e-04  1.71798447e-01  6.78144167e-01
    8.64555567e-01]
  [-1.31897472e-06  2.64371449e-03  4.42926195e-02  1.91425745e-01
    6.84820420e-01]
  [ 1.05715312e-05  6.82660595e-03  5.22889978e-03  6.56535860e-02
    9.02007279e-02]]

 [[-3.92732034e-02 -9.28227581e-02  2.74776757e-02  3.80198800

# Estimate the values $v$ of the states $\vec{q}$ under policy 2 (MSC)

In [5]:
# estimate the values v of the states q under policy 2 (maxSizeCenter)

def chooseServeNonzeroQueuesConfiguration(s, q):
    if q[2] == 0:
        s[0] = s[1] = 1
        return
    if q[0] == 0 or q[1] == 0:
        s[2] = 1
        return
    s[0] = s[1] = 1
    #chooseMaxWeightServiceConfiguration(s, q)

# initialize current states q, past average reward m, and value estimates v at time t = 0

q = np.array([0, 0, 0])
m = 0
v = np.zeros((c + 1, c + 1, c + 1))

# update v 10,000,000 times (with the first one being trivial)

for t in range(1, 10000001):
    # right now we've got q(t - 1)
    s = np.array([0, 0, 0])
    chooseServeNonzeroQueuesConfiguration(s, q) # compute service configuration s(t - 1) for the interval [t - 1, t]
    a = anArrivalVector(a1_avg, a2_avg, a3_avg) # generate arrival configuration a(t - 1) for the interval [t - 1, t]
    r = sum(q) # compute current reward r(t - 1)
    m_t_minus_1 = m # store m(t - 1)
    m = (1 - 1 / t) * m + (1 / t) * r # compute past average reward m(t) based on t, m(t - 1), and current reward r(t - 1)
    q_t_minus_1 = q # store q(t - 1)
    q = elementWiseMin(elementWiseMax(q + a - s, np.array([0, 0, 0])), np.array([c, c, c])) # compute current state q(t)
    
    # update value estimate v(q(t - 1)) based on v(q(t - 1)), t, r(t - 1), m(t - 1), and v(q(t))
    v[q_t_minus_1[0], q_t_minus_1[1], q_t_minus_1[2]] += (1 / t) * (r - m_t_minus_1 + v[q[0], q[1], q[2]] - v[q_t_minus_1[0], q_t_minus_1[1], q_t_minus_1[2]])

In [6]:
print('For MSC, the state-value estimates are summarized in')
print('v =\n', v, '\n')

For MSC, the state-value estimates are summarized in
v =
 [[[-2.45230825e+00 -1.56738034e+00 -3.39941787e-01 -1.60306882e-01
    4.41802813e-02]
  [-8.11796271e-01 -4.78158022e-01  1.31018945e-02  7.26024668e-02
    3.03177304e-01]
  [-3.16679183e-01 -8.06860615e-02  5.80752490e-02  4.55404765e-01
    3.83584094e-01]
  [-1.04954221e-01  2.22312687e-01  4.71293079e-01  3.00751176e-01
    3.62500173e-01]
  [ 1.69563055e-01  2.13784576e-01  3.53496996e-01  3.85231192e-01
    3.62806898e-01]]

 [[-9.19142693e-01 -4.08083441e-01 -2.69987528e-01  2.55172895e-02
    2.45564183e-01]
  [ 1.61575250e-02 -1.66146989e-01  3.72264486e-02  1.92170845e-01
    3.28678099e-01]
  [-3.89052804e-02  1.43710321e-02  1.04884223e-01  1.65555057e-01
    2.69003661e-01]
  [ 9.18464518e-02  4.63118550e-02  4.33043564e-01  1.56491302e-01
    2.16522835e-01]
  [ 2.78919654e-02  7.23355149e-02  1.26612517e-01  1.52447683e-01
    1.68798504e-01]]

 [[-4.01731072e-01 -2.13503475e-01  2.30459141e-02  2.14011685e-01
 